In [22]:
import os
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image

In [38]:
fractured_path = r"cervical fracture/Preprocessed_FracAtlas/Fractured"
nonfractured_path = r"cervical fracture/Preprocessed_FracAtlas/Non_fractured"
    
fractured_files = [os.path.join(fractured_path, file) for file in os.listdir(fractured_path)]
nonfractured_files = [os.path.join(nonfractured_path, file) for file in os.listdir(nonfractured_path)]


num_fractured_samples = len(fractured_files)
num_nonfractured_samples = len(nonfractured_files)

num_train_fractured_samples = int(0.7 * num_fractured_samples)
num_train_nonfractured_samples = int(0.7 * num_nonfractured_samples)
num_test_fractured_samples = int(0.3 * num_fractured_samples)
num_test_nonfractured_samples = int(0.3 * num_nonfractured_samples)


train_fractured_files = fractured_files[:num_train_fractured_samples]
train_nonfractured_files = nonfractured_files[:num_train_nonfractured_samples]
test_fractured_files = fractured_files[num_train_fractured_samples:]
test_nonfractured_files = nonfractured_files[num_train_nonfractured_samples:]


train_files = []
for file in train_fractured_files:
    train_files.append((file, 1.0))
for file in train_nonfractured_files:
    train_files.append((file, 0.0))
    

    
test_files = []
for file in test_fractured_files:
    test_files.append((file, 1.0))
for file in test_nonfractured_files:
    test_files.append((file, 0.0))

cervical fracture/Preprocessed_FracAtlas/Fractured\IMG0.jpg


In [39]:
train_df = pd.DataFrame(train_files, columns=['file_path','label'])
test_df = pd.DataFrame(test_files, columns=['file_path', 'label'])

In [40]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [41]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='file_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=4,
    class_mode='raw'  # Since labels are provided directly
)
validation_generator = val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='file_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=4,
    shuffle=True,
    class_mode='raw'  # Since labels are provided directly
)


Found 2857 validated image filenames.
Found 1226 validated image filenames.


In [42]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 gaussian_noise_4 (Gaussian  (None, 7, 7, 1280)        0         
 Noise)                                                          
                                                                 
 global_average_pooling2d_4  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_12 (Dense)            (None, 256)               327936    
                                                                 
 batch_normalization_8 (Bat  (None, 256)               1024      
 chNormalization)                                     

In [43]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall','AUC'])

In [44]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
lrp=ReduceLROnPlateau(monitor="val_accuracy", factor=0.1, patience=2)
filepath='best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
call=[checkpoint,lrp]
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch= 50,
    callbacks=call
    )

Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 0.7862 - accuracy: 0.5700 - precision: 0.1778 - recall: 0.5714 - auc: 0.5845
Epoch 1: val_accuracy improved from -inf to 0.36134, saving model to best_model.h5
50/50 [==============================] - 27s 499ms/step - loss: 0.7862 - accuracy: 0.5700 - precision: 0.1778 - recall: 0.5714 - auc: 0.5845 - val_loss: 0.9932 - val_accuracy: 0.3613 - val_precision: 0.1846 - val_recall: 0.7685 - val_auc: 0.5387 - lr: 0.0010


c:\Users\Ganesha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.6786 - accuracy: 0.6750 - precision: 0.2292 - recall: 0.2821 - auc: 0.5143
Epoch 2: val_accuracy improved from 0.36134 to 0.81566, saving model to best_model.h5
50/50 [==============================] - 19s 386ms/step - loss: 0.6786 - accuracy: 0.6750 - precision: 0.2292 - recall: 0.2821 - auc: 0.5143 - val_loss: 0.5462 - val_accuracy: 0.8157 - val_precision: 0.1429 - val_recall: 0.0093 - val_auc: 0.5093 - lr: 0.0010
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 0.5486 - accuracy: 0.7850 - precision: 0.2353 - recall: 0.1176 - auc: 0.5053
Epoch 3: val_accuracy improved from 0.81566 to 0.81811, saving model to best_model.h5
50/50 [==============================] - 19s 383ms/step - loss: 0.5486 - accuracy: 0.7850 - precision: 0.2353 - recall: 0.1176 - auc: 0.5053 - val_loss: 0.5363 - val_accuracy: 0.8181 - val_precision: 0.2667 - val_recall: 0.0185 - val_auc: 0.5839 - lr: 0.0010
Epoch 4/10
50/50 [==

In [45]:
model.evaluate(train_generator)


715/715 [==============================] - 54s 75ms/step - loss: 0.4351 - accuracy: 0.8138 - precision: 0.4543 - recall: 0.3074 - auc: 0.7281


[0.435066282749176,
 0.8137906789779663,
 0.45427727699279785,
 0.30738523602485657,
 0.7281434535980225]

In [46]:
model.evaluate(validation_generator)

307/307 [==============================] - 17s 55ms/step - loss: 0.5214 - accuracy: 0.7626 - precision: 0.2222 - recall: 0.1389 - auc: 0.5923


[0.521397054195404,
 0.7626427412033081,
 0.2222222238779068,
 0.1388888955116272,
 0.592349648475647]